
---

# Project: Part of Speech Tagging with Hidden Markov Models 

### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

<div align="center">
    <img src="images/post-hmm.png" width="800" height=auto>
    <figcaption>fig: Hidden Markov Model for Part of Speech Tagging</figcaption>
<div>
<br>

---

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
# %aimport helpers, tests

%aimport helpers
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
# from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

from pomegranate.hmm import DenseHMM
from pomegranate.distributions import Categorical
# Note: State and HiddenMarkovModel classes no longer exist


---

##### Step 1: Read and preprocess the dataset

We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 

$$
\begin{array}{lll}
\texttt{b100-38532} & & \text{\# Sentence identifier (unique ID)} \\
\texttt{Perhaps} & \texttt{ADV} & \text{\# word[TAB]tag pairs} \\
\texttt{it} & \texttt{PRON} & \\
\texttt{was} & \texttt{VERB} & \\
\texttt{right} & \texttt{ADJ} & \\
\texttt{;} & \texttt{.} & \\
\texttt{;} & \texttt{.} & \\
& & \\
& & \text{\# Blank line separates sentences} \\
\texttt{b100-35577} & & \text{\# Next sentence starts} \\
\texttt{...} & & \\
\end{array}
$$

---

##### Understanding the Dataset Structure and Preprocessing

##### What the Brown Corpus Is

The Brown corpus is a foundational dataset in computational linguistics containing over one million words of American English text from various sources (newspapers, novels, academic papers, etc.). For this part-of-speech tagging task, the corpus has been simplified to use the **Universal POS tagset**, which reduces the complexity from the original Penn Treebank's 45+ tags to just 12 universal categories:

- **NOUN** - nouns
- **VERB** - verbs  
- **ADJ** - adjectives
- **ADV** - adverbs
- **PRON** - pronouns
- **DET** - determiners
- **ADP** - adpositions (prepositions)
- **NUM** - numerals
- **CONJ** - conjunctions
- **PRT** - particles
- **.** - punctuation
- **X** - other/unknown

##### Dataset File Format Explanation

The corpus uses a specific plaintext format designed for sequence labeling tasks:


$$
\begin{array}{lll}
\texttt{b100-38532} & & \text{\# Sentence identifier (unique ID)} \\
\texttt{Perhaps} & \texttt{ADV} & \text{\# word[TAB]tag pairs} \\
\texttt{it} & \texttt{PRON} & \\
\texttt{was} & \texttt{VERB} & \\
\texttt{right} & \texttt{ADJ} & \\
\texttt{;} & \texttt{.} & \\
\texttt{;} & \texttt{.} & \\
& & \\
& & \text{\# Blank line separates sentences} \\
\texttt{b100-35577} & & \text{\# Next sentence starts} \\
\texttt{...} & & \\
\end{array}
$$

**Key Format Rules:**
- Each sentence begins with a unique identifier
- Word-tag pairs are separated by tabs (`\t`)
- One word-tag pair per line
- Sentences are separated by blank lines
- This format enables easy parsing for supervised learning


### Understanding the Dataset Loading Code

```python
data = Dataset("data/tags-universal.txt", "data/brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"
```

**What This Code Does:**

1. **Dataset Initialization**: Creates a Dataset object that:
   - Reads the tag vocabulary from `tags-universal.txt`
   - Parses the corpus from `data/brown-universal.txt`
   - Splits data into 80% training, 20% testing

2. **Data Validation**: The assertion ensures data integrity by verifying that:
   - Total sentences = Training sentences + Testing sentences
   - No data is lost during the splitting process

3. **Train-Test Split Strategy**: Uses temporal or sequential splitting (not random) to:
   - Preserve sentence ordering
   - Avoid data leakage between training and testing
   - Simulate real-world deployment conditions

---

In [9]:
def extract_tags_from_corpus(corpus_file_path):
    """Extract unique POS tags from the Brown corpus file."""
    import os
    
    # Check if tags file already exists
    if os.path.exists('data/tags-universal.txt'):
        return
        
    tags = set()
    
    with open(corpus_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            # Skip empty lines and sentence identifiers
            if line and '\t' in line:
                word, tag = line.split('\t')
                tags.add(tag)
    
    # Write tags to file in the data directory
    with open('data/tags-universal.txt', 'w', encoding='utf-8') as tag_file:
        for tag in sorted(tags):
            tag_file.write(tag + '\n')
    
    print(f"Found {len(tags)} unique tags: {sorted(tags)}")
    return sorted(tags)

# Generate the tags file
extract_tags_from_corpus("data/brown-universal.txt")


---

##### Understanding the Dataset Interface

The `Dataset` class provides a structured interface for accessing corpus data with built-in train-test splitting functionality. Both the main `Dataset` and its `Subset` components share a common API for consistent data access patterns.

### Class Hierarchy and Attributes

#### Dataset-Specific Attributes
```python
training_set    # Subset object containing training sentences (default: 80%)
testing_set     # Subset object containing testing sentences (default: 20%)
```

#### Shared Attributes (Dataset & Subset)
```python
sentences       # Dictionary mapping sentence_id -> Sentence(words, tags)
keys           # Immutable tuple of sentence identifiers in processing order
vocab          # Frozenset of unique words across all sentences
tagset         # Frozenset of unique POS tags used in the corpus
X              # Tuple of word sequences grouped by sentence
Y              # Tuple of tag sequences grouped by sentence (parallel to X)
N              # Total count of individual word tokens across all sentences
```

#### Common Methods
```python
stream()       # Iterator yielding flattened (word, tag) pairs across sentences
__iter__()     # Iterator over (sentence_id, Sentence) pairs
__len__()      # Number of sentences in the collection
```

### Data Structure Example

Consider a subset containing these sentences:
```python
sentences = {
    "s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")),
    "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))
}
```

The resulting subset attributes would be:

```python
subset.keys     # ("s0", "s1") - ordered tuple, not set
subset.vocab    # {"See", "Spot", "run", "ran"} - unordered frozenset
subset.tagset   # {"VERB", "NOUN"} - unordered frozenset
subset.X        # (("See", "Spot", "run"), ("Spot", "ran")) - matches key order
subset.Y        # (("VERB", "NOUN", "VERB"), ("NOUN", "VERB")) - parallel to X
subset.N        # 5 - total word tokens (3 + 2)
len(subset)     # 2 - number of sentences
```

### Key Design Principles

**Immutability**: Using frozenset and tuple types prevents accidental data modification during model training, ensuring reproducible results.

**Parallel Structure**: The `X` and `Y` attributes maintain identical ordering, enabling direct indexing correspondence between word and tag sequences.

**Efficient Access**: Pre-computed vocabulary and tag sets eliminate repeated scanning operations during feature extraction and model training phases.

**Consistent Interface**: Both `Dataset` and `Subset` implement the same methods, allowing interchangeable usage in training and evaluation code.

The interface design supports typical NLP workflows where you need access to both sentence-level structure (for sequence models) and flattened token streams (for vocabulary analysis and basic statistics).

---


In [10]:
data = Dataset("tags-universal.txt", "data/brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [11]:
key = 'b100-38532'

print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [8]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [12]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [13]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")

for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 


---

##### Step 2: Build a Most Frequent Class tagger

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

---

In [14]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    counts = defaultdict(lambda: defaultdict(int))
    for seq_A, seq_B in zip(sequences_A, sequences_B):
        for a, b in zip(seq_A, seq_B):
            counts[a][b] += 1
    return counts

# Calculate C(t_i, w_i)
emission_counts = pair_counts(data.training_set.Y, data.training_set.X)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [15]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

"""
Lightweight state representation for compatibility with Pomegranate model interface.

This namedtuple provides a minimal state object that mimics the structure of
Pomegranate HMM states while avoiding the complexity of full model instantiation.
Used primarily for interface compatibility in baseline models.

Attributes:
    name (str): Human-readable identifier for the state/tag
"""
FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    """
    Most Frequent Class baseline tagger for part-of-speech tagging evaluation.
    
    This baseline model assigns each word its most frequently observed POS tag
    from the training data. For unseen words, it returns a special MISSING state.
    The class provides a simplified interface compatible with more complex models
    like HMMs for fair performance comparison.
    
    The MFC approach represents a strong baseline because:
    - Many words have a dominant POS tag (e.g., "the" is almost always DET)
    - It captures lexical preferences without considering context
    - Performance degradation on unseen words highlights generalization challenges
    
    This baseline helps establish lower bounds for model performance and validates
    that more complex approaches provide meaningful improvements over simple
    frequency-based predictions.
    
    Attributes:
        missing (FakeState): Default state returned for unknown words
        table (defaultdict): Word-to-tag mapping with missing word fallback
    """
    
    """
    Class-level default state for handling out-of-vocabulary words.
    Using a class attribute ensures consistent behavior across all instances
    and provides a clear indicator for unseen word handling in predictions.
    """
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table: dict):
        """
        Initialize MFC tagger with pre-computed word-to-tag frequency mappings.
        
        Creates a lookup table that returns the most frequent tag for known words
        and a special MISSING state for unknown words. Uses defaultdict to handle
        missing keys gracefully without raising KeyError exceptions.
        
        Args:
            table (dict): Dictionary mapping words to their most frequent POS tags.
                Keys are word strings, values are tag strings representing the
                most commonly assigned tag for each word in training data.
                
        Example:
            >>> mfc_table = {"the": "DET", "cat": "NOUN", "runs": "VERB"}
            >>> tagger = MFCTagger(mfc_table)
            >>> print(tagger.table["the"].name)  # "DET"
            >>> print(tagger.table["unknown"].name)  # "<MISSING>"
        """
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq: list) -> tuple:
        """
        Generate POS tag predictions for a word sequence using most frequent class assignment.
        
        This method provides interface compatibility with Pomegranate HMM models by
        mimicking the viterbi() method signature and return format. However, instead
        of performing dynamic programming inference, it simply assigns the most
        frequent tag to each word independently.
        
        The method adds start and end tokens to match the expected format for
        sequence models, enabling direct comparison with HMM performance without
        interface modifications in evaluation code.
        
        Args:
            seq (list): Sequence of word tokens to be tagged.
                Should contain string tokens in the order they appear in the sentence.
                
        Returns:
            tuple: Two-element tuple containing:
                - float: Dummy log probability (always 0.0 for compatibility)
                - list: Enumerated sequence of (index, state) pairs where each state
                  is a FakeState object containing the predicted tag name. Includes
                  special start and end states at positions 0 and -1.
                  
        Example:
            >>> tagger = MFCTagger({"cat": "NOUN", "runs": "VERB"})
            >>> prob, states = tagger.viterbi(["cat", "runs"])
            >>> print(prob)  # 0.0
            >>> print([state.name for _, state in states])
            # ["<start>", "NOUN", "VERB", "<end>"]
            
        Note:
            The enumerated output format matches Pomegranate's viterbi interface
            where each element is (position, state). This enables seamless
            integration with existing evaluation pipelines designed for HMM models.
        """
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


"""
Build most frequent class lookup table from training data word-tag co-occurrence statistics.

This section computes the empirical frequency of each POS tag for every word in the
training corpus, then selects the most frequent tag as the default prediction for
each word. This approach captures the lexical preferences inherent in the language
while ignoring contextual information.

The frequency calculation process:
1. Count all (word, tag) co-occurrences in training data
2. For each word, identify the tag with maximum frequency
3. Store word -> most_frequent_tag mappings in lookup table

This baseline establishes performance expectations and validates that more complex
models provide meaningful improvements over simple frequency-based predictions.
"""

# Calculate word-tag co-occurrence frequencies across the training corpus
word_counts = pair_counts(data.training_set.X, data.training_set.Y)

"""
Create most frequent class lookup table by selecting the highest frequency tag for each word.

For each word in the training vocabulary, this loop identifies the POS tag that
appeared most frequently with that word and stores the mapping in mfc_table.
This represents the optimal prediction for each word under the assumption that
context doesn't matter and lexical frequency is the primary signal.

The max() function with key=tag_counts.get efficiently finds the tag with the
highest count for each word, handling ties deterministically based on the
internal ordering of the tag_counts dictionary.
"""
mfc_table = {}
for word, tag_counts in word_counts.items():
    mfc_table[word] = max(tag_counts, key=tag_counts.get)

# DO NOT MODIFY BELOW THIS LINE
"""
Instantiate the MFC baseline tagger with computed frequency table.

This creates the baseline model that will be compared against more sophisticated
approaches like Hidden Markov Models. The MFC tagger provides a strong but
simple baseline that captures lexical preferences without contextual reasoning.
"""
mfc_model = MFCTagger(mfc_table)

"""
Validation assertions to ensure correct MFC table construction.

These checks verify:
1. MFC table covers all words in training vocabulary (no missing entries)
2. All MFC table keys correspond to actual training vocabulary words
3. Expected number of out-of-vocabulary words in testing set (5521)

The assertions confirm that the baseline model is properly constructed and
that the train-test split has the expected vocabulary overlap characteristics.
"""
assert len(mfc_table) == len(data.training_set.vocab), "MFC table size must match training vocabulary"
assert all(k in data.training_set.vocab for k in mfc_table.keys()), "All MFC keys must be in training vocab"
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, "Expected 5521 OOV words in test set"

HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [17]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [18]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [19]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [20]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    counts = defaultdict(int)
    for sequence in sequences:
        for item in sequence:
            counts[item] += 1
    return counts

# Call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts(data.training_set.Y)

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%



---

##### Step 3: Build an HMM tagger

The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

---

In [21]:

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')


##### Bigram 

A **bigram** is a sequence of two adjacent elements in natural language processing and computational linguistics. In the context we've been discussing (POS tagging), bigrams refer to consecutive pairs of parts-of-speech tags or words.

###### Types of Bigrams

**Tag Bigrams**: Consecutive POS tag pairs
- Example: ("NOUN", "VERB") from "cat runs"
- Used for modeling grammatical transitions

**Word Bigrams**: Consecutive word pairs  
- Example: ("New", "York") or ("ice", "cream")
- Used for language modeling and phrase detection

###### Applications in NLP

**Hidden Markov Models**: Bigram counts estimate transition probabilities between states (POS tags):
$$P(\text{tag}_j | \text{tag}_i) = \frac{\text{count}(\text{tag}_i, \text{tag}_j)}{\text{count}(\text{tag}_i)}$$

**Language Modeling**: Bigrams help predict the next word based on the previous word:
$$P(\text{word}_j | \text{word}_i)$$

**Pattern Recognition**: Identify common linguistic constructions and collocations in text.

###### Example from Our Code Context

In the `bigram_counts` function we just documented, if you have tag sequences:
- `["DET", "NOUN", "VERB"]`
- `["NOUN", "VERB", "ADJ"]`

The bigrams would be:
- ("DET", "NOUN"): count = 1
- ("NOUN", "VERB"): count = 2  
- ("VERB", "ADJ"): count = 1

These counts are essential for training HMM models where you need to learn how likely one POS tag is to follow another in natural language.

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [22]:
def bigram_counts(sequences):
    """
    Calculate frequency counts for consecutive element pairs across multiple sequences.
    
    This function computes bigram statistics by counting all adjacent pairs of elements
    that appear in the input sequences. Bigrams are fundamental in natural language
    processing for modeling sequential dependencies, particularly useful for:
    - POS tag transition probabilities in Hidden Markov Models
    - Word co-occurrence patterns in language modeling
    - Sequential pattern analysis in any ordered data
    
    The function processes each sequence independently and accumulates counts across
    all sequences, providing corpus-level bigram statistics essential for probabilistic
    sequence models.
    
    Args:
        sequences (list): 2-dimensional array where each inner sequence contains
            ordered elements (strings, tags, tokens, etc.). Each sequence represents
            an independent ordered collection (e.g., sentences with POS tags).
            Example: [["NOUN", "VERB", "ADJ"], ["DET", "NOUN", "VERB"]]
            
    Returns:
        dict: Dictionary mapping bigram tuples to their occurrence counts.
            Keys are (element1, element2) tuples representing consecutive pairs.
            Values are integers representing frequency counts across all sequences.
            Uses defaultdict(int) for efficient counting without key existence checks.
            
    Example:
        >>> tag_sequences = [
        ...     ["NOUN", "VERB", "ADJ"],
        ...     ["NOUN", "VERB", "NOUN"],
        ...     ["DET", "NOUN", "VERB"]
        ... ]
        >>> counts = bigram_counts(tag_sequences)
        >>> print(counts[("NOUN", "VERB")])  # 3 (appears in all sequences)
        >>> print(counts[("VERB", "ADJ")])   # 1 (appears once)
        >>> print(counts[("DET", "NOUN")])   # 1 (appears once)
        
    Mathematical Foundation:
        For HMM training, bigram counts enable transition probability estimation:
        P(tag_j | tag_i) = count(tag_i, tag_j) / count(tag_i)
        
        Where bigram_counts provides the numerator count(tag_i, tag_j).
        
    Implementation Details:
        - Uses sliding window approach with range(len(sequence) - 1)
        - Handles sequences of any length, including single elements (no bigrams)
        - Empty sequences contribute no counts (graceful handling)
        - Memory efficient: O(unique_bigrams) space complexity
        
    Performance Considerations:
        - Time complexity: O(total_sequence_length) across all sequences
        - Space complexity: O(unique_bigrams) for the resulting dictionary
        - Uses defaultdict(int) to avoid repeated key existence checks
        
    Note:
        This function does not add special start/end tokens. For HMM applications
        requiring sentence boundary modeling, preprocess sequences to include
        "<start>" and "<end>" tokens before calling this function.
    """
    counts = defaultdict(int)
    for sequence in sequences:
        for i in range(len(sequence) - 1):
            counts[(sequence[i], sequence[i+1])] += 1
    return counts
    

# Call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(data.training_set.Y)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [23]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    counts = defaultdict(int)
    
    for sequence in sequences:
        if sequence:
            counts[sequence[0]] += 1
    return counts

# Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.training_set.Y)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [24]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    counts = defaultdict(int)
    for sequence in sequences:
        if sequence:
            counts[sequence[-1]] += 1
    return counts

# Calculate the count of each tag ending a sequence
tag_ends = ending_counts(data.training_set.Y)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [24]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# Create states with emission probability distributions P(word | tag) and add to the model
states = {}
for tag in data.training_set.tagset:
    emissions = {word: emission_counts[tag][word] / tag_unigrams[tag] for word in emission_counts[tag]}
    state = State(DiscreteDistribution(emissions), name=tag)
    states[tag] = state
    basic_model.add_state(state)

# Add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
for tag1 in data.training_set.tagset:
    # Add start transitions
    basic_model.add_transition(basic_model.start, states[tag1], tag_starts[tag1] / len(data.training_set))
    
    # Add end transitions
    basic_model.add_transition(states[tag1], basic_model.end, tag_ends[tag1] / tag_unigrams[tag1])
    
    # Add transitions between tags
    for tag2 in data.training_set.tagset:
        basic_model.add_transition(
            states[tag1],
            states[tag2],
            tag_bigrams[(tag1, tag2)] / tag_unigrams[tag1]
        )

# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [25]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_training_acc > 0.955, "Uh oh. Your HMM accuracy on the training set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 95.95%


### Example Decoding Sequences with the HMM Tagger

In [26]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [30]:
!!jupyter nbconvert --to html *.ipynb

['[NbConvertApp] Converting notebook hmm_tagger.ipynb to html',
 '[NbConvertApp] Writing 382865 bytes to hmm_tagger.html',
 '[NbConvertApp] Converting notebook hmm_warmup.ipynb to html',
 '[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).',
 '[NbConvertApp] Writing 334346 bytes to hmm_warmup.html']

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]